In [1]:
import pandas as pd

In [8]:
df= pd.read_csv('/content/drive/MyDrive/유진_나경_프로젝트 정리 폴더/환경 데이터 분석/2.EDA/predictfire_df.csv',index_col=0)

In [9]:
df.head()

,월,일,시간,읍면동,온도,시간단위풍향,시간단위풍속,습도,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,산불발생여부
0,1,1,오전,우지동,-0.7,288.5,1.3,79.0,1,2,2,30,0
1,1,1,오후,우지동,-2.3,274.6,2.0,58.0,1,2,2,30,0
2,1,2,오전,우지동,3.0,95.1,0.9,39.0,1,2,2,30,0
3,1,2,오후,우지동,-1.9,271.1,1.4,66.0,1,2,2,30,0
4,1,3,오전,우지동,0.0,180.2,0.6,74.0,1,2,2,30,0


In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
leddf = pd.DataFrame()
le.fit(df['읍면동'])
df['읍면동'] =le.transform(df['읍면동'])

In [12]:
timetoint = {'오전':0,'오후':1}
df['시간'] = df['시간'].replace(timetoint)

In [13]:
df = df.fillna(0)

In [14]:
import pandas as pd
import numpy as np

from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

In [15]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=['산불발생여부'])
y= df['산불발생여부']
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

In [16]:
nofire, fire = y_train.value_counts()
val_contamination = fire / nofire

In [17]:
model = IsolationForest(n_estimators=125, max_samples=len(X_train), contamination=val_contamination, random_state=42, verbose=0)
model.fit(X_train)

IsolationForest(contamination=0.004834162455973197, max_samples=128666,
                n_estimators=125, random_state=42)

In [18]:
def get_pred_label(model_pred):
    # IsolationForest 모델 출력
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

In [19]:
val_pred = model.predict(X_test) # model prediction
val_pred = get_pred_label(val_pred)

In [20]:
test = pd.DataFrame(y_test)
test.columns =['test']
test['test'].value_counts()

0    63099
1      274
Name: test, dtype: int64

In [21]:
pred = pd.DataFrame(val_pred)
pred.columns =['test']
pred['test'].value_counts()

0    63009
1      364
Name: test, dtype: int64

In [22]:
val_score = f1_score(y_test, val_pred, average='macro')

print(f'Validation F1 Score : [{val_score}]')
print(classification_report(y_test, val_pred))

Validation F1 Score : [0.7321942688721903]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     63099
           1       0.41      0.54      0.47       274

    accuracy                           0.99     63373
   macro avg       0.70      0.77      0.73     63373
weighted avg       1.00      0.99      1.00     63373

